# 05 - Conversor e Filtro para Submissão Final

**🎯 Objetivo deste Notebook:**

Este notebook serve como um passo final e documentado para formatar o arquivo de previsão gerado pelo `04-Final-Pipeline.ipynb`. Sua principal função é ler o arquivo Parquet completo, que contém previsões para todas as combinações de PDV/SKU (incluindo previsões de venda zero), e convertê-lo para o formato final exigido pela plataforma do Hackathon.

**✅ Processo de Conversão:**

1.  **Carregar Previsões:** Lê o arquivo `submission.parquet` da pasta `/submissions`, que contém o output completo do modelo.
2.  **Filtrar Vendas Zero:** Remove todas as linhas onde a coluna `quantidade` é igual a 0. Esta é uma exigência da plataforma, que limita o número de linhas do arquivo de submissão.
3.  **Renomear Colunas (Validação):** Garante que os nomes das colunas (`semana`, `pdv`, `produto`, `quantidade`) estão exatamente como o especificado no regulamento.
4.  **Salvar em CSV e Parquet:** Exporta o `DataFrame` filtrado para os arquivos finais (`submission_final.csv` e `submission_final.parquet`) prontos para o upload.

In [4]:
import pandas as pd
import os

print('📚 Bibliotecas carregadas com sucesso!')

📚 Bibliotecas carregadas com sucesso!


## 1. Definir Caminhos dos Arquivos

In [5]:
# Caminho para o arquivo de entrada gerado pelo pipeline 04
input_parquet_path = '../submissions/submission.parquet'

# Caminhos para os arquivos de saída finais que serão submetidos
output_csv_path = '../submissions/submission_final.csv'
output_parquet_path = '../submissions/submission_final.parquet'

print(f"Arquivo de entrada: {input_parquet_path}")
print(f"Arquivo de saída CSV: {output_csv_path}")
print(f"Arquivo de saída Parquet: {output_parquet_path}")

Arquivo de entrada: ../submissions/submission.parquet
Arquivo de saída CSV: ../submissions/submission_final.csv
Arquivo de saída Parquet: ../submissions/submission_final.parquet


## 2. Carregar e Processar o Arquivo de Previsão

In [6]:
print(f"🔄 Carregando o arquivo Parquet de previsões completas...")
if not os.path.exists(input_parquet_path):
    print(f"❌ Erro: Arquivo '{input_parquet_path}' não encontrado.")
    print("Certifique-se de executar o notebook '04-Final-Pipeline.ipynb' primeiro.")
else:
    # Carregar o DataFrame
    full_predictions_df = pd.read_parquet(input_parquet_path)
    print(f"✅ Arquivo carregado com sucesso. Shape original: {full_predictions_df.shape}")
    
    # --- PASSO 1: Garantir que os nomes das colunas estão corretos ---
    rename_map = {
        'pdv_id': 'pdv',
        'produto_id': 'produto'
    }
    columns_to_rename = [col for col in rename_map if col in full_predictions_df.columns]
    if columns_to_rename:
        print(f"Renomeando colunas: {columns_to_rename}")
        full_predictions_df.rename(columns=rename_map, inplace=True)
    
    # --- PASSO 2: Filtrar as linhas com previsão de quantidade zero ---
    print(f"\n🔧 Filtrando para manter apenas previsões com 'quantidade' > 0...")
    filtered_df = full_predictions_df[full_predictions_df['quantidade'] > 0].copy()
    
    linhas_removidas = len(full_predictions_df) - len(filtered_df)
    print(f"   • Linhas removidas: {linhas_removidas:,}")
    print(f"   • Shape final: {filtered_df.shape}")
    
    # --- PASSO 3: Validar os tipos de dados ---
    print("\n✔️ Validando e ajustando os tipos de dados para inteiros...")
    filtered_df['semana'] = filtered_df['semana'].astype(int)
    filtered_df['pdv'] = filtered_df['pdv'].astype(int)
    filtered_df['produto'] = filtered_df['produto'].astype(int)
    filtered_df['quantidade'] = filtered_df['quantidade'].astype(int)

    # --- PASSO 4: Salvar os arquivos finais ---
    print(f"\n💾 Salvando arquivos finais...")
    
    # Salvar em CSV
    filtered_df.to_csv(
        output_csv_path,
        sep=';',           
        index=False,       
        encoding='utf-8'   
    )
    print(f"   • Arquivo CSV salvo em '{output_csv_path}'")
    
    # Salvar em Parquet
    filtered_df.to_parquet(output_parquet_path, index=False)
    print(f"   • Arquivo Parquet salvo em '{output_parquet_path}'")
    
    print("\n🎉 Processo concluído! Os arquivos estão prontos para serem enviados.")
    
    # --- PASSO 5: Exibir um preview do arquivo final ---
    print("\n👀 Preview do DataFrame final:")
    display(filtered_df.head(10))

🔄 Carregando o arquivo Parquet de previsões completas...
✅ Arquivo carregado com sucesso. Shape original: (5221550, 4)

🔧 Filtrando para manter apenas previsões com 'quantidade' > 0...
   • Linhas removidas: 4,954,865
   • Shape final: (266685, 4)

✔️ Validando e ajustando os tipos de dados para inteiros...

💾 Salvando arquivos finais...
   • Arquivo CSV salvo em '../submissions/submission_final.csv'
   • Arquivo Parquet salvo em '../submissions/submission_final.parquet'

🎉 Processo concluído! Os arquivos estão prontos para serem enviados.

👀 Preview do DataFrame final:


,semana,pdv,produto,quantidade
69,1,1001371918471115422,2239307647969388381,4
76,1,1001371918471115422,2429796414123983930,1
88,1,1001371918471115422,3018796109558820617,1
99,1,1001371918471115422,3435249648591979862,1
103,1,1001371918471115422,3516587220187345473,1
107,1,1001371918471115422,3726736891643803768,1
112,1,1001371918471115422,3894706280449257667,7
133,1,1001371918471115422,4570988632387137438,4
139,1,1001371918471115422,4913890621258584650,1
151,1,1001371918471115422,5620797377392348813,1
